# Simple, Phase 1, approximate flight profile simulation

In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import control

# Purdue AA451 library
from scripts.jsbsim_utils import Logger, trim, simulate, linearize, rootlocus, clean_tf

In [76]:
aircraft_model = 'WaveRipper'

# Ground Trim

In [77]:
op_ground, props = trim(
    aircraft=aircraft_model,
    ic={
        'ic/vt-fps': 0,
        'ic/h-agl-ft': 5,
        'ic/psi-true-deg': 280,
        'fcs/left-brake-cmd-norm': 1,
        'fcs/right-brake-cmd-norm': 1,
        'fcs/center-brake-cmd-norm': 1,
    },
    design_vector=['ic/theta-rad', 'ic/h-agl-ft'],
    x0=[0, 0],
    verbose=True,
    method='Nelder-Mead', # works better with ground interaction
    ftol=1e-2,
)
op_ground

 final_simplex: (array([[-0.06305111,  0.17624348],
       [-0.06304254,  0.17621589],
       [-0.06305026,  0.17623362]]), array([476.00195821, 476.00200089, 476.00205581]))
           fun: 476.0019582057659
       message: 'Optimization terminated successfully.'
          nfev: 106
           nit: 57
        status: 0
       success: True
             x: array([-0.06305111,  0.17624348])



In file /Users/dev273/workfolder/boom/engine/aircraft/WaveRipper/WaveRipper.xml: line 155
<product> only has one argument which makes it a no-op.
Its argument will be evaluated but <product> will not be applied to the result.

In file /Users/dev273/workfolder/boom/engine/aircraft/WaveRipper/WaveRipper.xml: line 175
<product> only has one argument which makes it a no-op.
Its argument will be evaluated but <product> will not be applied to the result.

In file /Users/dev273/workfolder/boom/engine/aircraft/WaveRipper/WaveRipper.xml: line 186
<product> only has one argument which makes it a no-op.
Its argument will be evaluated but <product> will not be applied to the result.

In file /Users/dev273/workfolder/boom/engine/aircraft/WaveRipper/WaveRipper.xml: line 198
<product> only has one argument which makes it a no-op.
Its argument will be evaluated but <product> will not be applied to the result.

In file /Users/dev273/workfolder/boom/engine/aircraft/WaveRipper/WaveRipper.xml: line 208
<

RuntimeError: trim failed:
 final_simplex: (array([[-0.06305111,  0.17624348],
       [-0.06304254,  0.17621589],
       [-0.06305026,  0.17623362]]), array([476.00195821, 476.00200089, 476.00205581]))
           fun: 476.0019582057659
       message: 'Optimization terminated successfully.'
          nfev: 106
           nit: 57
        status: 0
       success: True
             x: array([-0.06305111,  0.17624348])
{'accelerations/a-pilot-x-ft_sec2': -1.1887984221204722, 'accelerations/a-pilot-y-ft_sec2': -1.5877559249554671, 'accelerations/a-pilot-z-ft_sec2': -21.075090116438524, 'accelerations/n-pilot-x-norm': -0.03694898452196417, 'accelerations/n-pilot-y-norm': -0.049348962788151554, 'accelerations/n-pilot-z-norm': -0.6550338257703153, 'accelerations/Nx': -0.03694898414513271, 'accelerations/Ny': 0.012089443581904002, 'accelerations/Nz': 0.698537845313642, 'accelerations/pdot-rad_sec2': 19.53253661033347, 'accelerations/qdot-rad_sec2': -0.6087917782359302, 'accelerations/rdot-rad_

In [78]:
op_ground

{'ic/vt-fps': 0,
 'ic/h-agl-ft': -0.01984969996708967,
 'ic/psi-true-deg': 280,
 'fcs/left-brake-cmd-norm': 1,
 'fcs/right-brake-cmd-norm': 1,
 'fcs/center-brake-cmd-norm': 1,
 'ic/theta-rad': -0.0039841619774816615}

In [79]:
log_ground = simulate(
    aircraft=aircraft_model,
    op_0=op_ground,
    tf=5,
    realtime=False, verbose=True)


In file /Users/dev273/workfolder/boom/engine/aircraft/WaveRipper/WaveRipper.xml: line 155
<product> only has one argument which makes it a no-op.
Its argument will be evaluated but <product> will not be applied to the result.

In file /Users/dev273/workfolder/boom/engine/aircraft/WaveRipper/WaveRipper.xml: line 175
<product> only has one argument which makes it a no-op.
Its argument will be evaluated but <product> will not be applied to the result.

In file /Users/dev273/workfolder/boom/engine/aircraft/WaveRipper/WaveRipper.xml: line 186
<product> only has one argument which makes it a no-op.
Its argument will be evaluated but <product> will not be applied to the result.

In file /Users/dev273/workfolder/boom/engine/aircraft/WaveRipper/WaveRipper.xml: line 198
<product> only has one argument which makes it a no-op.
Its argument will be evaluated but <product> will not be applied to the result.

In file /Users/dev273/workfolder/boom/engine/aircraft/WaveRipper/WaveRipper.xml: line 208
<

## Hover Trim

In [88]:
op_hover, props = trim(
    aircraft=aircraft_model,
    # For a list of initial conditions available:
    # https://jsbsim-team.github.io/jsbsim/classJSBSim_1_1FGInitialCondition.html
    ic={
        'ic/h-sl-ft': 650,      # Height above sea level initial condition in feet
        'ic/vt-fps': 500,       # True airspeed initial condition in feet/second
        'ic/psi-true-deg': 280, # Heading angle initial condition in degrees
    },
    eq_constraints = [
        lambda fdm: fdm['accelerations/udot-ft_sec2'],
        # lambda fdm: fdm['accelerations/vdot-ft_sec2'],
        lambda fdm: fdm['accelerations/wdot-ft_sec2'],
        # lambda fdm: fdm['accelerations/pdot-rad_sec2'],
    ],
    design_vector=[
        'fcs/throttle-cmd-norm',
        'fcs/elevator-cmd-norm',
    ],
    x0=[0.9],
    cost= lambda fdm: fdm['fcs/throttle-cmd-norm'],
    verbose=True,
    method='SLSQP',
    # method="cobyla",
    # bounds=[[0, 1]],
    bounds=[[0, 1], [-1, 1]],
)
op_hover

     fun: 0.9
     jac: array([1., 0.])
 message: 'Inequality constraints incompatible'
    nfev: 57
     nit: 6
    njev: 6
  status: 4
 success: False
       x: array([ 0.9       , -0.15950893])
constraint eq 42.751376462883385
constraint eq 28.344075612647305



In file /Users/dev273/workfolder/boom/engine/aircraft/WaveRipper/WaveRipper.xml: line 155
<product> only has one argument which makes it a no-op.
Its argument will be evaluated but <product> will not be applied to the result.

In file /Users/dev273/workfolder/boom/engine/aircraft/WaveRipper/WaveRipper.xml: line 175
<product> only has one argument which makes it a no-op.
Its argument will be evaluated but <product> will not be applied to the result.

In file /Users/dev273/workfolder/boom/engine/aircraft/WaveRipper/WaveRipper.xml: line 186
<product> only has one argument which makes it a no-op.
Its argument will be evaluated but <product> will not be applied to the result.

In file /Users/dev273/workfolder/boom/engine/aircraft/WaveRipper/WaveRipper.xml: line 198
<product> only has one argument which makes it a no-op.
Its argument will be evaluated but <product> will not be applied to the result.

In file /Users/dev273/workfolder/boom/engine/aircraft/WaveRipper/WaveRipper.xml: line 208
<

RuntimeError: trim failed:
     fun: 0.9
     jac: array([1., 0.])
 message: 'Inequality constraints incompatible'
    nfev: 57
     nit: 6
    njev: 6
  status: 4
 success: False
       x: array([ 0.9       , -0.15950893])
{'accelerations/a-pilot-x-ft_sec2': 42.751376007872054, 'accelerations/a-pilot-y-ft_sec2': -4.767527641235084, 'accelerations/a-pilot-z-ft_sec2': 12.3342143428962, 'accelerations/n-pilot-x-norm': 1.328753387466607, 'accelerations/n-pilot-y-norm': -0.1481792890587972, 'accelerations/n-pilot-z-norm': 0.383359101396987, 'accelerations/Nx': 1.3287533878399505, 'accelerations/Ny': -0.024112211633490973, 'accelerations/Nz': 0.12021376293816591, 'accelerations/pdot-rad_sec2': 39.14146800258483, 'accelerations/qdot-rad_sec2': -7.046958479424707, 'accelerations/rdot-rad_sec2': -1.736184781606401, 'accelerations/udot-ft_sec2': 42.751376462883385, 'accelerations/vdot-ft_sec2': -0.7285126257484911, 'accelerations/wdot-ft_sec2': 28.344075612647305, 'accelerations/gravity-ft_sec2

In [ ]:
log_hover = simulate(
    aircraft=aircraft_model,
    op_0=op_hover,
    tf=10,
    realtime=False)

In [ ]:
op_hover

In [ ]:
op_hover_auto = dict(op_hover)
op_hover_auto['ic/theta-deg'] = 0
op_hover_auto['ic/phi-deg'] = 0
op_hover_auto['ic/h-agl-ft'] = 60

op_hover_auto['ap/heading-cmd-deg'] = 280
op_hover_auto['ap/gear-enable'] = 1
op_hover_auto['ap/roll-enable'] = 0
op_hover_auto['ap/pitch-enable'] = 1
op_hover_auto['ap/yaw-enable'] = 0
op_hover_auto['ap/h-enable'] = 1
op_hover_auto['ap/h-sl-cmd-ft'] = 700

log_hover_auto = simulate(
    aircraft='F-35B-2',
    op_0=op_hover_auto,
    tf=50,
    realtime=False, verbose=True)

In [ ]:
log_hover_auto['fcs/throttle-pos-norm'].plot(label='0')
log_hover_auto['fcs/throttle-pos-norm[1]'].plot(label='1')
log_hover_auto['fcs/throttle-pos-norm[2]'].plot(label='2')
log_hover_auto['fcs/throttle-pos-norm[3]'].plot(label='3')

plt.legend()

In [ ]:
log_hover_auto['ap/h-error'].plot()

## Transition

## Cruise Trim

In [ ]:
op_cruise, props = trim(
    aircraft=aircraft_model,
    ic={
        'ic/gamma-deg': 0,
        'ic/vt-fps': 600,
        'ic/h-sl-ft': 10000,
        'gear/gear-cmd-norm': 0,
        'fcs/left-brake-cmd-norm': 0,
        'fcs/right-brake-cmd-norm': 0,
        'fcs/center-brake-cmd-norm': 0,
        'propulsion/engine/pitch-angle-rad': 0,
    },
    design_vector=[
        'fcs/throttle-cmd-norm',
        'fcs/elevator-cmd-norm',
        'fcs/rudder-cmd-norm',
        'fcs/aileron-cmd-norm',
        'ic/alpha-rad',
        'ic/beta-rad',
    ],
    method='SLSQP',
    eq_constraints= [
        lambda fdm: fdm['accelerations/udot-ft_sec2'],
        lambda fdm: fdm['accelerations/vdot-ft_sec2'],
        lambda fdm: fdm['accelerations/wdot-ft_sec2'],
        lambda fdm: fdm['accelerations/pdot-rad_sec2'],
        lambda fdm: fdm['accelerations/qdot-rad_sec2'],
        lambda fdm: fdm['accelerations/rdot-rad_sec2'],
    ],
    cost=lambda fdm: fdm['fcs/throttle-cmd-norm'],
    x0=[0.5, 0, 0, 0, 0, 0],
    verbose=False,
    bounds=[[0, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1]],
    tol=1e-3
)
op_cruise

In [ ]:
log_cruise = simulate(
    aircraft='F-35B-2',
    op_0=op_cruise,
    tf=10,
    realtime=False)

In [ ]:
op_cruise_auto = dict(op_cruise)
#op_cruise_auto['ic/theta-deg'] = 0
#op_cruise_auto['ic/phi-deg'] = 0
op_cruise_auto['ap/roll-enable'] = 1
op_cruise_auto['ap/pitch-enable'] = 1
op_cruise_auto['ap/yaw-enable'] = 1
op_cruise_auto['ap/h-enable'] = 1
op_cruise_auto['ap/heading-cmd-deg'] = 260
op_cruise_auto['ap/h-sl-cmd-ft'] = 10000

log_cruise_auto = simulate(
    aircraft='F-35B-2',
    op_0=op_cruise_auto,
    tf=40,
    realtime=False)